# vessel-cable-anomaly-hunter
DTU Deep Learning project 29, group 80

## Required Libraries Installation
Run this in your terminal before executing this:

In [ ]:
pip install -r requirements.txt

## 1) Data Download

#### File imports for the data download and preparation

In [1]:
import config
import src.data.ais_downloader as ais_downloader
import src.data.ais_filtering as ais_filtering
import src.data.ais_reader as ais_reader
import src.data.ais_to_parquet as ais_to_parquet

import src.data.ais_reader as ais_reader
import src.pre_proc.ais_query as ais_query 
import src.utils.ais_maps as ais_maps

/opt/anaconda3/envs/dl/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


#### Library imports for the data download and preparation

In [2]:
from tqdm import tqdm
from pathlib import Path
import pandas as pd
from datetime import date, timedelta

#### Set data preferences and configuration inputs

In [3]:
VERBOSE_MODE = config.VERBOSE_MODE                          # Whether to print verbose output

START_DATE = config.START_DATE                              # Start date for data downloading
END_DATE   = config.END_DATE                                # End date for data downloading

AIS_DATA_NAME = config.AIS_DATA_FOLDER                      # Name of the folder to store AIS data
DELETE_DOWNLOADED_CSV = config.DELETE_DOWNLOADED_CSV        # Whether to delete raw downloaded CSV files after processing

VESSEL_AIS_CLASS = config.VESSEL_AIS_CLASS                  # AIS classes of vessels to include

REMOVE_ZERO_SOG_VESSELS = config.REMOVE_ZERO_SOG_VESSELS    # Whether to remove vessels with zero Speed Over Ground
SOG_IN_MS = config.SOG_IN_MS                                # If True, SOG is in meters/second; if False, SOG is in knots
SOG_MIN_KNOTS = config.SOG_MIN_KNOTS                        # Minimum SOG in knots
SOG_MAX_KNOTS = config.SOG_MAX_KNOTS                        # Maximum SOG in knots

BBOX = config.BBOX                                          # Bounding Box to prefilter AIS data
POLYGON_COORDINATES = config.POLYGON_COORDINATES            # Polygon coordinates for filter Area of Interest

#### Create Directories and Paths

In [4]:
# --- Create paths ---
folder_path = Path(AIS_DATA_NAME)
folder_path.mkdir(parents=True, exist_ok=True)
csv_folder_path = folder_path / "csv"
csv_folder_path.mkdir(parents=True, exist_ok=True)
parquet_folder_path = folder_path / "parquet"
parquet_folder_path.mkdir(parents=True, exist_ok=True)

file_port_locations = folder_path / "port_locodes.csv" # Path to port locations file

#### Main Script
1) Download one single .csv AIS data file from http://aisdata.ais.dk (link to data column description http://aisdata.ais.dk/!_README_information_CSV_files.txt);
2) For a given AOI in Denmark with known cable positions, filter AIS messages by cleansing unrealistic/unphysical messages or duplicates and removes error-prone messages within port areas;
3) Segmentation of vessel tracks per MMSI and date;
4) Parquet Conversion;

In [5]:
# --- If you want to download all csv files before, uncomment the line below ---
# ais_downloader.download_multiple_ais_data(START_DATE, END_DATE, folder_path)

# --- Build the schedule of download string dates ---
dates = ais_downloader.get_work_dates(START_DATE, END_DATE, csv_folder_path, filter=False)

# --- Iterate with tqdm and download, unzip and delete ---
for day in tqdm(dates, desc=f"Processing data", unit="file" ):
    tag = f"{day:%Y-%m}" if day < date.fromisoformat("2024-03-01") else f"{day:%Y-%m-%d}"
    print(f"\nProcessing date: {tag}")

    # --- Download one day ---
    csv_path = ais_downloader.download_one_ais_data(day, csv_folder_path)
    
    # --- Load CSV into DataFrame ---
    df_raw = ais_reader.read_single_ais_df(csv_path, BBOX, verbose=VERBOSE_MODE)
    # --- Optionally delete the downloaded CSV file ---
    if DELETE_DOWNLOADED_CSV: csv_path.unlink(missing_ok=True)
    
    # --- Filter and split ---
    # Filter AIS data, keeping Class A and Class B by default,
    df_filtered = ais_filtering.filter_ais_df(
        df_raw,                                               # raw AIS DataFrame
        polygon_coords=POLYGON_COORDINATES,                   # polygon coordinates for precise AOI filtering
        allowed_mobile_types=VESSEL_AIS_CLASS,                # vessel AIS class filter
        apply_polygon_filter=True,                            # keep polygon filtering enabled boolean
        remove_zero_sog_vessels=REMOVE_ZERO_SOG_VESSELS,      # use True/False to enable/disable 90% zero-SOG removal
        output_sog_in_ms=SOG_IN_MS,                           # convert SOG from knots in m/s (default) boolean
        sog_min_knots=SOG_MIN_KNOTS,                          # min SOG in knots to keep (None to disable)
        sog_max_knots=SOG_MAX_KNOTS,                          # max SOG in knots to keep (None to disable) 
        port_locodes_path=file_port_locations,                # path to port locodes CSV
        exclude_ports=True,                                   # exclude port areas boolean 
        verbose=VERBOSE_MODE,                                 # verbose mode boolean
    )
        
    # --- Parquet conversion ---
    # Save to Parquet by MMSI
    ais_to_parquet.save_by_mmsi(
        df_filtered,                                             # filtered AIS DataFrame 
        verbose=VERBOSE_MODE,                                    # verbose mode boolean
        output_folder=parquet_folder_path                        # output folder path
    )

Processing data:   0%|          | 0/3 [00:00<?, ?file/s]


Processing date: 2025-08-01
Skipping 2025-08-01 download: already present in ais-data/csv folder
 Read AIS data: 1,128,873 rows within bbox,  511 unique vessels
 [filter_ais_df] Before filtering: 1,128,873 rows, 511 vessels
 [filter_ais_df] Type filtering: 1,093,101 rows (removed 35,772) using ['Class A', 'Class B']
 [filter_ais_df] MMSI filtering: 1,093,079 rows, 508 vessels
 [filter_ais_df] Duplicate removal: 638,467 rows, 508 vessels
 [filter_ais_df] Polygon filtering: 337,847 rows (removed 300,620), 378 vessels
 [filter_ais_df] Port-area removal: removed 160,778 rows in 3 overlapping ports
 [filter_ais_df] COG sanity: 175,598 rows (removed 1,471) with range [0, 360] deg
 [filter_ais_df] SOG sanity: 173,215 rows (removed 2,381) with range [0.5, 35.0] knots
 [filter_ais_df] Final: 173,215 rows, 327 unique vessels (SOG in m/s)


Processing data:  33%|███▎      | 1/3 [00:04<00:08,  4.21s/file]

 [save_by_mmsi] Parquet dataset written/appended at: /Users/federicomarra/Documents/GitHub/dl-dark-vessel-hunter/ais-data/parquet

Processing date: 2025-08-02
Skipping 2025-08-02 download: already present in ais-data/csv folder
 Read AIS data: 1,161,214 rows within bbox,  452 unique vessels
 [filter_ais_df] Before filtering: 1,161,214 rows, 452 vessels
 [filter_ais_df] Type filtering: 1,120,127 rows (removed 41,087) using ['Class A', 'Class B']
 [filter_ais_df] MMSI filtering: 1,114,486 rows, 447 vessels
 [filter_ais_df] Duplicate removal: 616,865 rows, 447 vessels
 [filter_ais_df] Polygon filtering: 314,494 rows (removed 302,371), 342 vessels
 [filter_ais_df] Port-area removal: removed 168,854 rows in 3 overlapping ports
 [filter_ais_df] COG sanity: 145,303 rows (removed 337) with range [0, 360] deg
 [filter_ais_df] SOG sanity: 144,010 rows (removed 1,291) with range [0.5, 35.0] knots
 [filter_ais_df] Final: 144,010 rows, 290 unique vessels (SOG in m/s)


Processing data:  67%|██████▋   | 2/3 [00:08<00:04,  4.47s/file]

 [save_by_mmsi] Parquet dataset written/appended at: /Users/federicomarra/Documents/GitHub/dl-dark-vessel-hunter/ais-data/parquet

Processing date: 2025-08-03
Skipping 2025-08-03 download: already present in ais-data/csv folder
 Read AIS data: 1,079,891 rows within bbox,  399 unique vessels
 [filter_ais_df] Before filtering: 1,079,891 rows, 399 vessels
 [filter_ais_df] Type filtering: 1,043,284 rows (removed 36,607) using ['Class A', 'Class B']
 [filter_ais_df] MMSI filtering: 1,043,284 rows, 398 vessels
 [filter_ais_df] Duplicate removal: 599,515 rows, 398 vessels
 [filter_ais_df] Polygon filtering: 319,478 rows (removed 280,037), 288 vessels
 [filter_ais_df] Port-area removal: removed 165,183 rows in 3 overlapping ports
 [filter_ais_df] COG sanity: 153,909 rows (removed 386) with range [0, 360] deg
 [filter_ais_df] SOG sanity: 152,641 rows (removed 1,266) with range [0.5, 35.0] knots
 [filter_ais_df] Final: 152,641 rows, 248 unique vessels (SOG in m/s)


Processing data: 100%|██████████| 3/3 [00:13<00:00,  4.45s/file]

 [save_by_mmsi] Parquet dataset written/appended at: /Users/federicomarra/Documents/GitHub/dl-dark-vessel-hunter/ais-data/parquet


## 2) Pre-processing

#### File imports for the pre-processing

In [ ]:
import config
import src.pre_proc.pre_processing_utils as pre_processing_utils
import src.pre_proc.ais_query as ais_query

#### Library imports for the pre-processing

In [ ]:
from pathlib import Path
import pandas as pd
import json

#### Set pre-processing preferences and input configuration

In [ ]:
AIS_DATA_NAME = config.AIS_DATA_FOLDER
folder_path = Path(AIS_DATA_NAME)
parquet_folder_path = folder_path / "parquet"

SEGMENT_MAX_LENGTH = 300  # datapoints

NUMERIC_COLS = config.NUMERIC_COLS
# if u want to do it withouth a end date comment next line
TRAIN_START_DATE = "2025-10-20"
TRAIN_END_DATE = "2025-11-08"

TEST_START_DATE = "2025-11-09"
TEST_END_DATE = "2025-11-10"

#### Pre processing function

In [ ]:
def main_pre_processing(dataframe_type: str = "all"):

    if dataframe_type == "all":
        main_pre_processing("train")
        main_pre_processing("test")
        return
        
    elif dataframe_type == "train":
        print(f"[pre_processing] Querying AIS data for training period: {TRAIN_START_DATE} to {TRAIN_END_DATE}")
        # Loading filtered data from parquet files
        dates = (
            pd.date_range(TRAIN_START_DATE, TRAIN_END_DATE, freq="D")
            .strftime("%Y-%m-%d")
            .tolist()
        )
        df = ais_query.query_ais_duckdb(parquet_folder_path, dates=dates, verbose=VERBOSE_MODE)
        
    elif dataframe_type == "test":
        print(f"[pre_processing] Querying AIS data for testing period: {TEST_START_DATE} to {TEST_END_DATE}")
        # Loading filtered data from parquet files
        dates = (
            pd.date_range(TEST_START_DATE, TEST_END_DATE, freq="D")
            .strftime("%Y-%m-%d")
            .tolist()
        )
        df = ais_query.query_ais_duckdb(parquet_folder_path, dates=dates, verbose=VERBOSE_MODE)
    else:
        raise ValueError(f"Invalid dataframe_type: {dataframe_type}. Must be 'train' or 'test'.")
    

    # Dropping unnecessary columns and rows with missing values
    print(f"[pre_processing] Initial data size: {len(df)} records.")
    print(f"[pre_processing] Dropping unnecessary columns and rows with missing values...")
    df.drop(columns=[ 
        'Type of mobile', 
        'ROT', 
        'Heading', 
        'IMO', 
        'Callsign', 
        'Name', 
        'Navigational status',
        'Cargo type', 
        'Width', 
        'Length',
        'Type of position fixing device', 
        'Draught', 
        'Destination', 
        'ETA',
        'Data source type', 
        'A', 'B', 'C', 'D', 
        'Date'], inplace=True, errors='ignore')

    df.dropna(inplace=True)
    print(f"[pre_processing] Data size after dropping: {len(df)} records.")

    # Grouping Ship types
    commercial_types = ["Cargo", "Tanker"]
    passenger_types = ["Passenger", "Pleasure", "Sailing"]
    service_types = ["Dredging", "Law enforcement", "Military", "Port tender", "SAR", "Towing", "Towing long/wide","Tug"]
    valid_types =  ["Fishing", "Service", "Commercial", "Passenger"]

    df.loc[df["Ship type"].isin(commercial_types), "Ship type"] = "Commercial"
    df.loc[df["Ship type"].isin(passenger_types), "Ship type"] = "Passenger"
    df.loc[df["Ship type"].isin(service_types), "Ship type"] = "Service"
    df.loc[~df["Ship type"].isin(valid_types), "Ship type"] = "Other"

    print("[pre_processing] Ship type counts:")
    print(df["Ship type"].value_counts())

    # Adding △T feature
    df = pre_processing_utils.add_delta_t(df)
    df.drop(columns=["DeltaT"], inplace=True)

    # Splitting segments
    print(f"[pre_processing] Splitting segments to max length {SEGMENT_MAX_LENGTH}...")
    df = pre_processing_utils.split_segments_fixed_length(df, max_len=SEGMENT_MAX_LENGTH)

    # Normalizing numeric columns
    df, mean, std = pre_processing_utils.normalize_df(df, NUMERIC_COLS)

    # Encoding Navicational Status as one-hot
    #df, nav_status_to_id = pre_processing_utils.one_hot_encode_nav_status(df)

    # Ship type labeling (mapping to be used later)
    df, ship_type_to_id = pre_processing_utils.label_ship_types(df)
    
    # Saving pre-processed DataFrame
    if dataframe_type == "train":
        print(f"[pre_processing] Saving pre-processed DataFrame to {config.PRE_PROCESSING_DF_TRAIN_PATH}...")
        output_path = config.PRE_PROCESSING_DF_TRAIN_PATH
        metadata_path = config.PRE_PROCESSING_METADATA_TRAIN_PATH
    else:
        print(f"[pre_processing] Saving pre-processed DataFrame to {config.PRE_PROCESSING_DF_TEST_PATH}...")
        output_path = config.PRE_PROCESSING_DF_TEST_PATH
        metadata_path = config.PRE_PROCESSING_METADATA_TEST_PATH

    print(f"[pre_processing] Columns of pre-processed DataFrame:\n{df.columns.tolist()}")
    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    df.to_parquet(output_path, index=False)

    # Saving preprocessing metadata
    print(f"[pre_processing] Saving preprocessing metadata to {metadata_path}...")
    meta = {
        "mean": mean.tolist(),
        "std": std.tolist(),
        #"nav_status_to_id": nav_status_to_id,
        "ship_type_to_id": ship_type_to_id
    }

    with open(metadata_path, "w") as f:
        json.dump(meta, f, indent=4)

#### Pre processing script

In [ ]:
main_pre_processing("train")
main_pre_processing("test")

## Training

#### File imports for the training

#### Library imports for the training

#### Set training preferences

#### Script

## Evaluation

#### File imports for the evaluation

#### Library imports for the evaluation

#### Set evaluation preferences

#### Script

## Inspection